<style>
code {
    color:red !important;
}
</style>

# Explore

Explore is a class which implements automatic persistent memoization. You can easily declare an experiment as a directed acyclic graph (DAG) where the nodes are functions and the edges represent variables that are produced and consumed by functions.

During the first execution of the graph, variables are persisted to the disk which implies a longer graph execution time (due to variable loading and saving). However, for future executions, if the node and all the sub-functions called within the node remain unchanged, the results will simply be retrieved.

## Setup

We clear and setup paths.

In [46]:
rootFolder = [pwd filesep '..' filesep '..'];
warning('off');
rmpath(genpath([rootFolder filesep 'test']));
rmpath(genpath([rootFolder filesep 'example']));
warning('on');
addpath(genpath([rootFolder filesep 'src']));
addpath(genpath([rootFolder filesep 'example' filesep 'fcn']));

## Session and Context

You need to define a *session* by its name. This allows you to retrieve the session from other notebooks or experiments. A session contains the history of the launched nodes of the dataflow graph. The session is created **inside** the context of the *calling function*, here `nbsession()`. If the calling function path or file name is changed, a new *context* will be created and the associated sessions will be created again. Therefore, once created, the calling function should not be modified.

In [47]:
sess = 'sess1';
explo = nbsession(sess);

Explore folder "C:\Users\jahsue\explo"
Retrieve existing work session "sess1" for context "nbsession_26a0b20876a1f9a2e4df2a996232c94a"



For the same context, you can define session-dependent nodes. A concrete example is to switch the data node, i.e., node that provides input data for the whole experiment, depending on the goal of your experiment. For instance, you might want to use a small subset of your entire data set if your are debugging your code. In *production*, you would like to use the entire data set. Here, `'sess1'` would output the small data set and `'sess2'` the entire data set.

In [48]:
switch sess
    case 'sess1'
        fcn = @fcnTestexp1;
    case 'sess2'
        fcn = @fcnTestexp1bis;
end

## Pipe

You need to define the pipes you will use. A pipe is a way to identify the data that is put in it. If another node needs the data to compute its code, it checks the signature of the data before loading it. It compares this signature with the signature used during the last computation. If the code and its dependencies did not change and the input signatures did not change, the node code will not be computed, but the results will be retrieved.

Everytime you create a pipe, you need to define its signature type. For the moment, you have the choice between 3 different types:
1. **Matfile** (`'matfile'`): it hashes the content of the Matlab file payload (excluding the header) and save it as signature. A description of MAT-file format (including header and payload definition) can be found [here](https://maxwell.ict.griffith.edu.au/spl/matlab-page/matfile_format.pdf). The `-v6` option is used to save the variables.
2. **Date** (`'date'`): it signs with the Matlab file save date. This kind of signature is suited for big data files. The `'-v7.3'` option is used to save the variables.
3. **Auto** (`'auto'`): depending on the size of the variable, the `'matfile'` (> 2GB) or the `'date'` signature mode is selected. This is the preferred option to prepare for experiment with growing data.

In [49]:
explo.addPip('s1','matfile');
explo.addPip('s2','date');
explo.addPip('s3','auto');

## Variable
You can declare variables that will be used to parametrize the graph computations, e.g., number of iterations. Here we are defining a variable called `'v1'` with the `'auto'` signature mode.

In [50]:
explo.addVar('v1','s3');

## Node
Here, we define the nodes of the graph. For instance, the node `'m2'` is a wrapper to the function `@fcnTestexp2` which has 3 inputs and 1 output. The output `'s1'` indicates that the first output will be signed using the `'matfile'` mode. The second input `'m1_s1'` indicates that the second input of the function `@fcnTestexp2` will receive the first output of the node `'m1'`. 

One **important** thing is that the character `'_'` is reserved, because the variable full name follows the syntax `'<variableLabel>_<pipeLabel>'`.

Finally, you can assign a class to the node (by default `'branch'`):
- `'branch'`: this is the default class, e.g., `'m2'`. The nodes belonging to this class will be evaluated by default when running an experiment.
- `'root'`: the nodes belonging to this class, will not be evaluated by default when running an experiment. E.g., the nodes responsible to retrieve data from other servers should typically belong to this class, avoiding fetching the data everytime.
- `'leaf'`: the nodes belonging to this class, e.g., `'m3'`, typically implement plotting function. Therefore, it is possible to view all the plots of an experiment when evaluating only this class of nodes. Functionally, this class is equivalent to the `'branch'`class.

*Note: the node concept is equivalent to the Python decorator concept.*

In [51]:
explo.addFcn('m1',fcn,{},{'s1','s2'},'class','branch');
explo.addFcn('m2',@fcnTestexp2,{'m1_s2','m1_s1','v1_s3'},{'s1'});
explo.addFcn('m3',@fcnTestexp3,{'m1_s1','m2_s1'},{'s3'},'class','leaf');

## Init

To build a graph, call the `.init()` method. You can optionally visualize the graph with the `'plot'` option (by default `false`).

**If you plot the graph in a Jupyter notebook**, the magic command `%plot native` should be called prior to that to ensure the plot is not closed at the end of the command (closing the plot and putting the image of the graph in the the notebook is the standard behavior `%plot`). In that case, you will not be able to observe the progress of the computations.

Once the method `.init()` is launched, i.e., the graph is built, the graph structure (variables, nodes, ...) cannot be modified. 

*Note: This compilation step is equivalent to the static graph session concept in TensorFlow.*

In [52]:
%plot native
explo.init('plot',true);

## Run
You can assign the value `3` to the variable `'v1_s3'` and then run the graph to the end node `'m3'`. In the meanwhile, take a look at the Matlab figure. When all nodes are evaluated, you can retrieve the value of `'m3_s3'`.

In [53]:
explo.setVariable('v1_s3',3);
explo.run('-e:m3');
data = explo.getVariable('m3_s3');

Action     = Save auto [matfile-v6] output "v1_s3" (started on 15-Apr-2019 17:08:28)...
             |--- Elapsed time is 0.0099384 seconds.
Action     = Compute signature of output "v1_s3" (started on 15-Apr-2019 17:08:28)...
             |--- Elapsed time is 0.010074 seconds.
--- Function @fcnTestexp1 "m1" <branch>
Task       =   First test function
Input(s)   = [  ]
Output(s)  = [ m1_s1 | m1_s2 ]
Action     = Compute function signature (started on 15-Apr-2019 17:08:28)...
             |--- Elapsed time is 0.034443 seconds.
Decision   = RETRIEVE (same signatures)...
--- Function @fcnTestexp2 "m2" <branch>
Task       =   Second test function
Input(s)   = [ m1_s2 | m1_s1 | v1_s3 ]
Output(s)  = [ m2_s1 ]
Action     = Compute function signature (started on 15-Apr-2019 17:08:28)...
             |--- Elapsed time is 0.024172 seconds.
Decision   = RETRIEVE (same signatures)...
--- Function @fcnTestexp3 "m3" <leaf>
Task       =   Third test function
Input(s)   = [ m1_s1 | m2_s1 ]
Output(s)  

If you run the graph again `explo.run('-e:m3')` without changing the nodes, the results are retrieved and not re-computed and the execution time is significantly reduced.

In [ ]:
explo.run('-e:m3');

## Sub-function Awareness

If you modify the function `@fcnTestexp2sub` (which is a sub-function called in `@fcnTestexp2`), the corresponding node `'m2'` is re-computed. However, since the output of this node (which is used as input for the node `'m3'`) did not changed, the results of the node `'m3'` are simply retrieved.

In [ ]:
file = which('fcnTestexp2sub');
fid = fopen(file,'a'); fprintf(fid,'%s',' '); fclose(fid); % Add empty character at the end
explo.run('-e:m3');

## Figure

After you init the graph with `explo.init('plot',true)`, you can view different types of information to let you know about the status of the computational graph.


- **Status of computations** (after `.run()`):<br>
1) A node or edge is gray when the computation is not planned<br>
2) A node or edge is blue when it is planned but not computed yet<br>
3) A node or edge is orange when it is being computed at the current time<br>
4) A node or edge is green when the computation is finished


- **Status of functions** (after `.run()` and after computation or retrieval):<br>
1) A function label is green when the result is retrieved<br>
2) A function label is orange when the function content has changed and maybe also the inputs<br>
3) A function label is blue hwne the function content is the same but inputs have changed<br>
4) A function label is red when it was forced


- **Information on function arguments** (on the figure):<br>
You can view information on the function arguments by clicking on the nodes with the tooltip.


- **Shape of the nodes** (on the figure):<br>
1) A circle node represents a `'branch'` node.<br>
2) A diamond node represents a `'leaf'` node.<br>
3) A square node represents a `'root'` node.

    



## Run Examples

You can also uncomment the following lines to run the nodes with another configuration.

In [ ]:
% -- Class evaluation: Run all the nodes belonging to class 'root'
% explo.run('-c:root');

% -- End node evaluation: Run only the nodes needed to compute the end node 'm3'
% explo.run('-e:m3');

% -- Single node evaluation: Run only the node 'm3'
% explo.run('-s:m3');

% -- Run only the nodes needed to compute end node and force their computation, i.e., force mode
% explo.run('-e:m3-m:f');

% -- Run all 'leaf' nodes (if the plotting functions are in these nodes, this will retrieve all the plots)
% explo.run('-c:leaf');

## Context Information
You can display the context information like the history of the commands, the size of the variables on disk, the function input/outputs, etc.

In [ ]:
% -- Get the experiment information
explo.info()

% -- Open the Matlab file in Matlab IDE
% edit('fcnTestexp2.m')

% -- Retrieve its absolute path
% which('fcnTestexp2.m')

## More on Contexts and Sessions

An experiment is declared using the command `explo = Explore().session('sess')`.
    
* You can create multiple sessions for each context. The context is defined by the script or function full path where `Explore()` is called or the command line (when called from command line). All the caches and persistency will be defined and saved in the session `'sess'`. If you rename or move the calling script/function file, it will automatically create a new context. Changing sessions will allow to switch to different variable caches. This can be used for instance when switching from debug (small data set) to production (bigger one).

* You can emulate an experiment creation or retrieval from another context (script, function or the command line) with the following commands: <br>
1) `explo = Explore('')` emulates the expriment from the command line context<br>
2) `explo = Explore('~/experiments/exp1.m')` emulates the experiment from the script `~/experiments/exp1.m` context<br>
3) `explo = Explore(@exp1)` emulates the experiment from the script handle `@exp1` context<br>
